## Fannie Proof of Concept Model

In [86]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd

In [87]:
df = pd.read_csv(r'Data\1-Fannie_Mae_Multifamily.csv')

C:\Users\samvi\AppData\Local\Temp\ipykernel_40980\1241587058.py:1: DtypeWarning: Columns (12,26,29,34,35,36,37,38,39,40,46,49,51,53,54,55,56,61) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'Data\1-Fannie_Mae_Multifamily.csv')


In [88]:
#Find the loans which have final events either before or after the reporting periods. 
df['Liquidation/Prepayment Date'] = pd.to_datetime(df['Liquidation/Prepayment Date'])
df['Reporting Period Date'] = pd.to_datetime(df['Reporting Period Date'])
min_reporting = df.groupby('Loan Number')['Reporting Period Date'].min()
max_reporting = df.groupby('Loan Number')['Reporting Period Date'].max()
final_lp_date = df.groupby('Loan Number')['Liquidation/Prepayment Date'].max()
too_early = final_lp_date < min_reporting
too_late = final_lp_date > max_reporting
early_loans = final_lp_date[too_early].index.tolist()
late_loans = final_lp_date[too_late].index.tolist()

In [89]:
#Remove the loans which have the final event logged as before the last reporting date (only 2 loans out of 69k)
df = df[~df['Loan Number'].isin(early_loans)]

In [90]:
#Adding the start period information for surival analysis 
df = df.sort_values(['Loan Number', 'Reporting Period Date'])
first_dates = df.groupby('Loan Number')['Reporting Period Date'].transform('min')
def months_between(start, end):
    return (end.dt.year - start.dt.year) * 12 + (end.dt.month - start.dt.month)
df['start'] = months_between(first_dates, df['Reporting Period Date'])

In [91]:
#Adding the stop period information for survival analysis 
df = df.sort_values(['Loan Number', 'start'])
df['stop'] = df.groupby('Loan Number')['start'].shift(-1)
df['stop'] = df['stop'].fillna(df['start'] + 1)

In [92]:
df['start'] = df['start'].astype(int)
df['stop'] = df['stop'].astype(int)

In [93]:
#Setting up state column
df = df.sort_values(['Loan Number', 'Reporting Period Date'])
df['state'] = 'Current'

#Adding in default information to state column
mask = (df['Loan Payment Status'].notna()) & (df['Loan Payment Status'] != 'Current')
df.loc[mask, 'state'] = 'Default'

In [94]:
#Dropping event types we don't like 
codes_to_drop = {'Fully Paid, Refinanced', 'Third Party Sale', 'Other Liquidation', 'Dissolution', 'Repurchase', 'Deed-in-Lieu', 'Discounted Payoff', 'Substitution'}

loans_to_drop = df[df['Liquidation/Prepayment Code'].isin(codes_to_drop)]['Loan Number'].unique()
df = df[~df['Loan Number'].isin(loans_to_drop)]

In [95]:
df.loc[df['Foreclosure Date'].notna(), 'Liquidation/Prepayment Date'] = df['Foreclosure Date']
df['Liquidation/Prepayment Date'] = pd.to_datetime(df['Liquidation/Prepayment Date'])
df['event_time'] = months_between(first_dates, df['Liquidation/Prepayment Date'])
df['event_time'] = df['event_time'].astype('Int64')

In [96]:
#Making more rows for when the event happened past the reporting window 

non_null_events = df[df['event_time'].notna()]

# Step 2: Identify loans where event_time > all start values
loans_to_copy = []
for loan_num, group in non_null_events.groupby('Loan Number'):
    event_time = group['event_time'].iloc[0]
    all_starts = df[df['Loan Number'] == loan_num]['start']
    if (event_time > all_starts).all():
        loans_to_copy.append((loan_num, event_time))

# Step 3: Duplicate row with max start and modify start/stop
new_rows = []
for loan_num, event_time in loans_to_copy:
    loan_group = df[df['Loan Number'] == loan_num]
    max_idx = loan_group['start'].idxmax()
    row_to_copy = df.loc[max_idx].copy()

    row_to_copy['start'] = row_to_copy['start'] + 1
    row_to_copy['stop'] = event_time + 1

    new_rows.append(row_to_copy)

# Step 4: Append to the original DataFrame
if new_rows:
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

In [97]:
#Logging the events when they happened (state column being populated)

df = df.copy()  # Work on a copy to avoid altering original

# Step 1: Find loans with non-null event_time
loans_with_event = df[df['event_time'].notnull()]['Loan Number'].unique()

for loan in loans_with_event:
    loan_rows = df[df['Loan Number'] == loan]

    # Step 2: Get the consistent non-null event_time and code for the loan
    event_time_vals = loan_rows['event_time'].dropna().unique()
    code_vals = loan_rows['Liquidation/Prepayment Code'].dropna().unique()

    # Defensive checks
    if len(event_time_vals) == 0 or len(code_vals) == 0:
        continue  # Skip this loan if any values are missing

    event_time = event_time_vals[0]
    code = code_vals[0]

    # Step 3: Find the row with largest start <= event_time
    valid_rows = loan_rows[loan_rows['start'] <= event_time]
    if not valid_rows.empty:
        target_start = valid_rows['start'].max()
        mask = (df['Loan Number'] == loan) & (df['start'] == target_start)
        df.loc[mask, 'state'] = code

In [98]:
end_states = ['Fully Paid, Prepaid', 'Fully Paid, Matured', 'Foreclosure']

df = df.sort_values(by=['Loan Number', 'start'])

def truncate_loan(loan_df):
    for i, row in loan_df.iterrows():
        if row['state'] in end_states:
            return loan_df.loc[:i]  # include the row with the terminal state
    return loan_df  # if no terminal state found, return all rows

df = df.groupby('Loan Number', group_keys=False).apply(truncate_loan).reset_index(drop=True)

C:\Users\samvi\AppData\Local\Temp\ipykernel_40980\390653705.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Loan Number', group_keys=False).apply(truncate_loan).reset_index(drop=True)


In [ ]:
df.to_csv(r'Data\fannie_cleaned.csv', index=False)

# Testing

In [85]:
df[df['Loan Number'] == 1682079906].tail(10)

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,start,stop,state,event_time
835,1682079906,2002-08-01,2002-07-12,2012-11-01,"$2,205,000.00",Amortizing Balloon,Fixed,DUS,"$2,205,000.00",360.0,7.285,Second,385373,2002-08-01,80.0,1.25,Lender UW DSCR,123,NaN,NaN,Y,Pari Passu,NaN,1.0,780.0,Multifamily,1978,HOUSTON,TX,77074,"HOUSTON-THE WOODLANDS-SUGAR LAND, TX METROPOLI...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2008-02-01,1.0,7.285,2012-11-01,"$2,082,773.67",NaN,Current,N,NaN,N,NaN,"YM(117), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,75.0,N,NaN,66,67,Current,<NA>
836,1682079906,2002-08-01,2002-07-12,2012-11-01,"$2,205,000.00",Amortizing Balloon,Fixed,DUS,"$2,205,000.00",360.0,7.285,Second,385373,2002-08-01,80.0,1.25,Lender UW DSCR,123,NaN,NaN,Y,Pari Passu,NaN,1.0,780.0,Multifamily,1978,HOUSTON,TX,77074,"HOUSTON-THE WOODLANDS-SUGAR LAND, TX METROPOLI...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2008-03-01,1.0,7.285,2012-11-01,"$2,079,902.00",NaN,Current,N,NaN,N,NaN,"YM(117), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,75.0,N,NaN,67,68,Current,<NA>
837,1682079906,2002-08-01,2002-07-12,2012-11-01,"$2,205,000.00",Amortizing Balloon,Fixed,DUS,"$2,205,000.00",360.0,7.285,Second,385373,2002-08-01,80.0,1.25,Lender UW DSCR,123,NaN,NaN,Y,Pari Passu,NaN,1.0,780.0,Multifamily,1978,HOUSTON,TX,77074,"HOUSTON-THE WOODLANDS-SUGAR LAND, TX METROPOLI...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2008-04-01,1.0,7.285,2012-11-01,"$2,077,855.26",NaN,Current,N,NaN,N,NaN,"YM(117), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,75.0,N,NaN,68,69,Current,<NA>
838,1682079906,2002-08-01,2002-07-12,2012-11-01,"$2,205,000.00",Amortizing Balloon,Fixed,DUS,"$2,205,000.00",360.0,7.285,Second,385373,2002-08-01,80.0,1.25,Lender UW DSCR,123,NaN,NaN,Y,Pari Passu,NaN,1.0,780.0,Multifamily,1978,HOUSTON,TX,77074,"HOUSTON-THE WOODLANDS-SUGAR LAND, TX METROPOLI...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2008-05-01,1.0,7.285,2012-11-01,"$2,075,375.20",NaN,Current,N,NaN,N,NaN,"YM(117), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,75.0,N,NaN,69,70,Current,<NA>
839,1682079906,2002-08-01,2002-07-12,2012-11-01,"$2,205,000.00",Amortizing Balloon,Fixed,DUS,"$2,205,000.00",360.0,7.285,Second,385373,2002-08-01,80.0,1.25,Lender UW DSCR,123,NaN,NaN,Y,Pari Passu,NaN,1.0,780.0,Multifamily,1978,HOUSTON,TX,77074,"HOUSTON-THE WOODLANDS-SUGAR LAND, TX METROPOLI...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2008-06-01,1.0,7.285,2012-11-01,"$2,073,300.06",NaN,Current,N,NaN,N,NaN,"YM(117), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,75.0,N,NaN,70,71,Current,<NA>
840,1682079906,2002-08-01,2002-07-12,2012-11-01,"$2,205,000.00",Amortizing Balloon,Fixed,DUS,"$2,205,000.00",360.0,7.285,Second,385373,2002-08-01,80.0,1.25,Lender UW DSCR,123,NaN,NaN,Y,Pari Passu,NaN,1.0,780.0,Multifamily,1978

In [31]:
filtered_df['Loan Number'].unique()

array([1674109600, 1674470958, 1675021925, 1675083135, 1675244426,
       1675291003, 1675683837, 1675811072, 1676257521, 1676513023,
       1680249163, 1681289048, 1682079906, 1683660407, 1688229861,
       1693764600, 1694950389, 1696251332, 1717461087, 1717463779,
       1717463780, 1717463795, 1717474927, 1717480224, 1717482762,
       8000984133, 8000984134, 8000984156, 8000984782, 8000984904,
       8000984964, 8000984988, 8000984997, 8000985042, 8000985339,
       8000985400, 8000985850, 8000985993, 8000986444, 8000987324,
       8000987565, 8000987895, 8000988077, 8000988104, 8000989370,
       8000989371, 8000990236, 8000993682, 8300001556, 8300002281,
       8300006176])

In [46]:
pd.set_option('display.max_columns', None)
df[df['Loan Number'] == 1717470045].tail(10)

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,start,stop,state,event_time


# Old Stuff

In [83]:
df['Liquidation/Prepayment Code'] = df['Liquidation/Prepayment Code'].replace('nan', np.nan)

# Drop exact duplicates just in case
df = df.drop_duplicates()

# For each loan, count how many *distinct* non-null codes exist
code_counts = (
    df.dropna(subset=['Liquidation/Prepayment Code'])  # keep only rows where the code is not null
      .groupby('Loan Number')['Liquidation/Prepayment Code']
      .nunique()
)

# Filter for loans with >1 distinct non-null code
multiple_code_loans = code_counts[code_counts > 1]

# Get the loan numbers
loan_numbers = multiple_code_loans.index.tolist()

print(f"Number of loans with >1 distinct non-null code: {len(loan_numbers)}")
print(loan_numbers)

Number of loans with >1 distinct non-null code: 0
[]


In [78]:
df['Loan Number'].nunique()

68144

In [70]:
non_null_codes = df[df['Liquidation/Prepayment Code'].notna()]

# Count how many non-null codes each loan has
code_counts = non_null_codes.groupby('Loan Number')['Liquidation/Prepayment Code'].count()

# Get loan numbers where there is more than one non-null code
loan_numbers = code_counts[code_counts > 1].index.tolist()

print(loan_numbers)
print(len(loan_numbers))

[1673867584, 1673867585, 1673887828, 1673893029, 1673893030, 1673901049, 1673901050, 1673904357, 1673904358, 1673904359, 1673904360, 1673904361, 1673908755, 1673908756, 1673908757, 1673908762, 1673908763, 1673908764, 1673916780, 1673916781, 1673916782, 1673916783, 1673916784, 1673937676, 1673941561, 1673941562, 1673946426, 1673946428, 1673946429, 1673946430, 1673946431, 1673950957, 1673950958, 1673954057, 1673954058, 1673958626, 1673958627, 1673958628, 1673958629, 1673961311, 1673961312, 1673961313, 1673961314, 1673961315, 1673961316, 1673966038, 1673966039, 1673966040, 1673966041, 1673982437, 1673982438, 1673994188, 1673994189, 1673994190, 1674006282, 1674016679, 1674020837, 1674020838, 1674020839, 1674020840, 1674020841, 1674020842, 1674024428, 1674027808, 1674027809, 1674029884, 1674029885, 1674029886, 1674029887, 1674029888, 1674032876, 1674035588, 1674035589, 1674037837, 1674039027, 1674039028, 1674042639, 1674042640, 1674044964, 1674047584, 1674047585, 1674047586, 1674047587, 167

In [21]:
#Helper functions for dealing with liquidation/prepayment code 
def months_between(start, end):
    return (end.year - start.year) * 12 + (end.month - start.month)

def deal_with_drops(drops, df):
    print(len(drops))
    df = df[~df['Loan Number'].isin(drops)]
    print(len(df))
    return df

'''
def deal_with_foreclosures(foreclosures, df):
    for loan_num in foreclosures:
        loan_rows = df[df['Loan Number'] == loan_num].copy()

        min_date = loan_rows['Reporting Period Date'].min()
        max_date = loan_rows['Reporting Period Date'].max() + MonthEnd(1)  # extend max by 1 month

        event_date = foreclosures[loan_num] # assumed pre-parsed datetime

        if min_date <= event_date <= max_date:
            # Inside Reporting Window
            mask = loan_rows['Reporting Period Date'] <= event_date
            valid_rows = loan_rows[mask]
            if not valid_rows.empty:
                target_date = valid_rows['Reporting Period Date'].max()

                # Tag the 'state' column at this reporting date
                df.loc[
                    (df['Loan Number'] == loan_num) & (df['Reporting Period Date'] == target_date),
                    'state'
                ] = 'Foreclosed'
        else:
            # Outside Reporting Window
            latest_row = loan_rows.iloc[-1].copy()
            latest_date = latest_row['Reporting Period Date']
            latest_month_end = latest_row['stop']

            new_row = latest_row.copy()
            new_row['Reporting Period Date'] = latest_date + MonthEnd(1)

            new_row['state'] = 'Foreclosed'
            new_row['start'] = latest_month_end
            new_row['stop'] = months_between(latest_date, event_date)

            df.loc[len(df)] = new_row  # appends the new row
    print(len(df))
    return df
'''

def deal_with_foreclosures(foreclosures, df):
    df = df.sort_values(['Loan Number', 'Reporting Period Date'])
    df.set_index(['Loan Number', 'Reporting Period Date'], inplace=True)

    rows_to_drop = []
    new_rows = []

    for loan_num, event_date in foreclosures.items():
        try:
            loan_rows = df.loc[loan_num]
        except KeyError:
            continue

        if isinstance(loan_rows, pd.Series):  # Only one row for this loan
            loan_rows = loan_rows.to_frame().T

        dates = loan_rows.index.get_level_values('Reporting Period Date')
        if dates.empty:
            continue

        min_date = dates.min()
        max_date = dates.max() + MonthEnd(1)

        if min_date <= event_date <= max_date:
            # Find latest date before or equal to event
            valid_dates = dates[dates <= event_date]
            if valid_dates.empty:
                continue
            target_date = valid_dates.max()

            df.loc[(loan_num, target_date), 'state'] = 'Foreclosed'

            # Drop all later rows
            drop_dates = dates[dates > target_date]
            rows_to_drop.extend([(loan_num, d) for d in drop_dates])

        else:
            # Outside window → append a new row
            latest_date = dates.max()
            latest_row = loan_rows.loc[latest_date].copy()

            new_index = (loan_num, latest_date + MonthEnd(1))
            stop = event_date
            start = latest_row.get('stop', latest_date + MonthEnd(1))

            new_row = latest_row.copy()
            new_row['state'] = 'Foreclosed'
            new_row['start'] = start
            new_row['stop'] = stop

            new_rows.append((new_index, new_row))

    # Drop rows after foreclosed
    if rows_to_drop:
        df = df.drop(index=rows_to_drop)

    # Append new rows
    if new_rows:
        new_df = pd.DataFrame(
            [row for _, row in new_rows],
            index=pd.MultiIndex.from_tuples([idx for idx, _ in new_rows], names=['Loan Number', 'Reporting Period Date'])
        )
        df = pd.concat([df, new_df])

    return df.reset_index()
        

def deal_with_others(others, df):
    for loan_num in others:
        loan_rows = df[df['Loan Number'] == loan_num].copy()

        min_date = loan_rows['Reporting Period Date'].min()
        max_date = loan_rows['Reporting Period Date'].max() + MonthEnd(1)  # extend max by 1 month

        event_date = others[loan_num][0] # assumed pre-parsed datetime
        event_type = others[loan_num][1]

        if min_date <= event_date <= max_date:
            # Inside Reporting Window
            mask = loan_rows['Reporting Period Date'] <= event_date
            valid_rows = loan_rows[mask]
            if not valid_rows.empty:
                target_date = valid_rows['Reporting Period Date'].max()

                # Tag the 'state' column at this reporting date
                df.loc[
                    (df['Loan Number'] == loan_num) & (df['Reporting Period Date'] == target_date),
                    'state'
                ] = event_type
        else:
            # Outside Reporting Window
            latest_row = loan_rows.iloc[-1].copy()
            latest_date = latest_row['Reporting Period Date']
            latest_month_end = latest_row['stop']

            new_row = latest_row.copy()
            new_row['Reporting Period Date'] = latest_date + MonthEnd(1)

            new_row['state'] = event_type
            new_row['start'] = latest_month_end
            new_row['stop'] = months_between(latest_date, event_date)
            
            df.loc[len(df)] = new_row  # appends the new row
    print(len(df))
    return df
        

In [22]:
loan_numbers_default = df[df['Liquidation/Prepayment Code'] == 'Foreclosure']['Loan Number'].unique()
print(loan_numbers_default)

[1673908757 1674044964 1674047591 1674086447 1674107624 1674146231
 1674199885 1674470958 1674494372 1674663660 1674715082 1674752072
 1674777477 1674835106 1674876515 1674880753 1675021925 1675120248
 1675129862 1675129863 1675129864 1675146259 1675182391 1675244426
 1675259070 1675262794 1675279414 1675405789 1675446847 1675536445
 1675646353 1675656856 1675656859 1675666765 1675683837 1675718163
 1675746317 1675766617 1675805588 1675810184 1675811072 1675824884
 1675875405 1675953908 1675985491 1675990929 1676051769 1676097542
 1676113420 1676373481 1676527924 1676917930 1676943579 1676970905
 1677011403 1677049752 1677077177 1677129661 1677208037 1677229942
 1677229943 1677229944 1677229945 1677229946 1677610318 1678322242
 1678409152 1678409153 1678465413 1678488758 1678520836 1679391928
 1679430382 1679694576 1679905185 1679905187 1679946148 1679997485
 1680249163 1681141333 1681243735 1681298093 1681353081 1681367843
 1681568029 1681568030 1682079906 1683329255 1684361394 168442

In [23]:
#Putting Liquidation/Prepayment Code info into state column. NEEDS TO BE CHANGED MUCH LATER TO CAPTURE ALL POSSIBILITIES
state_ending = ['Fully Paid, Prepaid', 'Fully Paid, Matured', 'Foreclosure']
df['Liquidation/Prepayment Code'] = df['Liquidation/Prepayment Code'].astype(str)
df['Foreclosure Date'] = pd.to_datetime(df['Foreclosure Date'])
df = df.sort_values(['Loan Number', 'Reporting Period Date'])

loans_to_drop = [] #loan numbers
foreclosures = {} #loan number: foreclosure date
others = {} #loan number: event date


for loan_id, loan_df in df.groupby('Loan Number'):
    loan_df = loan_df.sort_values('Reporting Period Date')
    
    for idx, row in loan_df.iterrows():
        event_status = str(row['Liquidation/Prepayment Code'])
        loan_number = row['Loan Number']

        if event_status.lower() in ['nan', 'none', ''] or pd.isna(row['Liquidation/Prepayment Code']):
            continue  # skip if it's empty
        
        if event_status in state_ending:
            if event_status == 'Foreclosure':
                foreclosures[loan_number] = row['Foreclosure Date']
            else:
                others[loan_number] = (row['Liquidation/Prepayment Date'], event_status)
        else:
            loans_to_drop.append(loan_number)
        break

'''
for loan_id, loan_df in df.groupby('Loan Number'): #df loan doop
    loan_df = loan_df.sort_values('Reporting Period Date') 
    
    for idx, row in loan_df.iterrows(): #individual loan loop
        event_status = str(row['Liquidation/Prepayment Code'])
        loan_number = row['Loan Number']
        if event_status:
            if event_status in state_ending:
                if event_status == 'Foreclosure':
                    foreclosures[row['Loan Number']] = row['Foreclosure Date']
                else:
                    others[row['Loan Number']] = (row['Liquidation/Prepayment Date'], row['Liquidation/Prepayment Code'])
            else:
                loans_to_drop.append(row['Loan Number'])
            break
'''

df = deal_with_drops(loans_to_drop, df)
df = deal_with_foreclosures(foreclosures, df)
df = deal_with_others(others, df)

933
4885898


C:\Users\samvi\AppData\Local\Temp\ipykernel_36924\1044989806.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = new_row  # appends the new row
C:\Users\samvi\AppData\Local\Temp\ipykernel_36924\1044989806.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = new_row  # appends the new row
C:\Users\samvi\AppData\Local\Temp\ipykernel_36924\1044989806.py:153: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a

KeyboardInterrupt: 

In [11]:
df

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,...,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,start,event_time,stop,state


In [52]:
df['Liquidation/Prepayment Code'].value_counts()

Liquidation/Prepayment Code
Fully Paid, Prepaid       35849
Fully Paid, Matured        3529
Fully Paid, Refinanced      671
Foreclosure                 642
Third Party Sale             84
Other Liquidation            69
Dissolution                  41
Repurchase                   28
Deed-in-Lieu                 27
Discounted Payoff            12
Substitution                  1
Name: count, dtype: int64

### Working stuff

In [19]:
df['Liquidation/Prepayment Code'].value_counts()

Liquidation/Prepayment Code
Fully Paid, Prepaid       35849
Fully Paid, Matured        3529
Fully Paid, Refinanced      671
Foreclosure                 642
Third Party Sale             84
Other Liquidation            69
Dissolution                  41
Repurchase                   28
Deed-in-Lieu                 27
Discounted Payoff            12
Substitution                  1
Name: count, dtype: int64

In [50]:
pd.set_option('display.max_columns', None)
df[df['Loan Number'] == 1700953876]

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,start,event_time,stop,state
597408,1700953876,2006-04-27,2006-03-15,2016-04-01,"$800,000.00",Amortizing Balloon,Fixed,DUS,"$800,000.00",360.0,6.71,First,873520,2006-04-01,67.2,1.28,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,40.0,Multifamily,1966,LINCOLN PARK,MI,48146,"DETROIT-WARREN-DEARBORN, MI METROPOLITAN STATI...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2006-04-01,1.0,6.71,2016-04-01,"$800,000.00",NaN,Current,N,NaN,N,NaN,"YM(114), 1%(3), O*(3)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,67.0,N,NaN,0,<NA>,1,Current
597402,1700953876,2006-04-27,2006-03-15,2016-04-01,"$800,000.00",Amortizing Balloon,Fixed,DUS,"$800,000.00",360.0,6.71,First,873520,2006-04-01,67.2,1.28,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,40.0,Multifamily,1966,LINCOLN PARK,MI,48146,"DETROIT-WARREN-DEARBORN, MI METROPOLITAN STATI...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2006-05-01,1.0,6.71,2016-04-01,"$799,305.80",NaN,Current,N,NaN,N,NaN,"YM(114), 1%(3), O*(3)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,67.0,N,NaN,1,<NA>,2,Current
597373,1700953876,2006-04-27,2006-03-15,2016-04-01,"$800,000.00",Amortizing Balloon,Fixed,DUS,"$800,000.00",360.0,6.71,First,873520,2006-04-01,67.2,1.28,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,40.0,Multifamily,1966,LINCOLN PARK,MI,48146,"DETROIT-WARREN-DEARBORN, MI METROPOLITAN STATI...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2006-06-01,1.0,6.71,2016-04-01,"$798,756.70",NaN,Current,N,NaN,N,NaN,"YM(114), 1%(3), O*(3)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,67.0,N,NaN,2,<NA>,3,Current
597369,1700953876,2006-04-27,2006-03-15,2016-04-01,"$800,000.00",Amortizing Balloon,Fixed,DUS,"$800,000.00",360.0,6.71,First,873520,2006-04-01,67.2,1.28,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,40.0,Multifamily,1966,LINCOLN PARK,MI,48146,"DETROIT-WARREN-DEARBORN, MI METROPOLITAN STATI...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2006-07-01,1.0,6.71,2016-04-01,"$798,055.55",NaN,Current,N,NaN,N,NaN,"YM(114), 1%(3), O*(3)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,67.0,N,NaN,3,<NA>,4,Current
597395,1700953876,2006-04-27,2006-03-15,2016-04-01,"$800,000.00",Amortizing Balloon,Fixed,DUS,"$800,000.00",360.0,6.71,First,873520,2006-04-01,67.2,1.28,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,40.0,Multifamily,1966,LINCOLN PARK,MI,48146,"DETROIT-WARREN-DEARBORN, MI METROPOLITAN STATI...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2006-08-01,1.0,6.71,2016-04-01,"$797,499.23",NaN,Current,N,NaN,N,NaN,"YM(114), 1%(3), O*(3)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,67.0,N,NaN,4,<NA>,5,Current
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [51]:
def has_gap(group):
    expected = set(range(group['start'].max() + 1))
    actual = set(group['start'])
    return expected != actual

# Step 2: Group by loan and apply the gap check
loans_with_gaps = df.groupby('Loan Number').filter(has_gap)

# Step 3: Get unique loan numbers with gaps
bad_loan_list = loans_with_gaps['Loan Number'].unique().tolist()
bad_loan_list

[1677229943,
 1700953876,
 1704866745,
 1717459050,
 8000985965,
 8000986649,
 8000990404,
 8300000321,
 8300001556]

In [55]:
df[df['Loan Number'] == 8000986649]

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,start,stop
4019285,8000986649,2003-03-31,2003-03-04,2013-04-01,"$2,933,000.00",Amortizing Balloon,Fixed,DUS,"$2,933,000.00",360.0,5.51,First,NaN,NaN,70.0,1.3,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,83.0,Multifamily,1950,TEMPLE HILLS,MD,20748,"WASHINGTON-ARLINGTON-ALEXANDRIA, DC-VA-MD-WV M...",83.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Non-REO,2003-03-01,1.0,5.51,2013-04-01,"$2,933,000.00",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,98.0,NaN,NaN,0,1.0
4019242,8000986649,2003-03-31,2003-03-04,2013-04-01,"$2,933,000.00",Amortizing Balloon,Fixed,DUS,"$2,933,000.00",360.0,5.51,First,NaN,NaN,70.0,1.3,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,83.0,Multifamily,1950,TEMPLE HILLS,MD,20748,"WASHINGTON-ARLINGTON-ALEXANDRIA, DC-VA-MD-WV M...",83.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Non-REO,2003-04-01,1.0,5.51,2013-04-01,"$2,933,000.00",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,98.0,NaN,NaN,1,2.0
4019295,8000986649,2003-03-31,2003-03-04,2013-04-01,"$2,933,000.00",Amortizing Balloon,Fixed,DUS,"$2,933,000.00",360.0,5.51,First,NaN,NaN,70.0,1.3,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,83.0,Multifamily,1950,TEMPLE HILLS,MD,20748,"WASHINGTON-ARLINGTON-ALEXANDRIA, DC-VA-MD-WV M...",83.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Non-REO,2003-05-01,1.0,5.51,2013-04-01,"$2,929,795.70",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,98.0,NaN,NaN,2,3.0
4019325,8000986649,2003-03-31,2003-03-04,2013-04-01,"$2,933,000.00",Amortizing Balloon,Fixed,DUS,"$2,933,000.00",360.0,5.51,First,NaN,NaN,70.0,1.3,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,83.0,Multifamily,1950,TEMPLE HILLS,MD,20748,"WASHINGTON-ARLINGTON-ALEXANDRIA, DC-VA-MD-WV M...",83.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Non-REO,2003-06-01,1.0,5.51,2013-04-01,"$2,927,025.11",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,98.0,NaN,NaN,3,4.0
4019231,8000986649,2003-03-31,2003-03-04,2013-04-01,"$2,933,000.00",Amortizing Balloon,Fixed,DUS,"$2,933,000.00",360.0,5.51,First,NaN,NaN,70.0,1.3,Lender UW DSCR,120,NaN,NaN,Y,NaN,NaN,1.0,83.0,Multifamily,1950,TEMPLE HILLS,MD,20748,"WASHINGTON-ARLINGTON-ALEXANDRIA, DC-VA-MD-WV M...",83.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,Non-REO,2003-07-01,1.0,5.51,2013-04-01,"$2,923,793.37",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Involuntary Prepayment,Standard,98.0,NaN,NaN,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019332,8000986649,2003-03-31,2003-03-04,2013-04-01,"$2,933,000.00",Amortizing

In [48]:
df.head(30)

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,start
56,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-01-01,1.0,7.650,2009-12-01,"$14,725,385.83",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN,0
29,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-02-01,1.0,7.650,2009-12-01,"$14,714,704.00",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN,1
28,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-03-01,1.0,7.650,2009-12-01,"$14,703,954.09",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN,2
51,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-04-01,1.0,7.650,2009-12-01,"$14,693,135.65",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN,3
54,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-05-01,1.0,7.650,2009-12-01,"$14,682,248.26",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN,4
31,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS

## FOOLING AROUND

In [11]:
bad_loans = df.groupby('Loan Number')['Loan Payment Status'].apply(lambda x: not (x == 'Current').all())
defaulters = bad_loans[bad_loans].index.tolist()

In [12]:
has_lp_event = df.groupby('Loan Number')['Liquidation/Prepayment Code'].apply(lambda x: x.notna().any())
eventers = has_lp_event[has_lp_event].index.tolist()

In [13]:
true_defaulters = [x for x in defaulters if x not in eventers]

In [15]:
false_defaulters = [x for x in defaulters if x in eventers]

In [21]:
df_new= df[df['Loan Number'].isin(false_defaulters)]

In [40]:
df_new['Loan Payment Status'].value_counts()

Loan Payment Status
Current                  165712
90+ Days Delinquent       10640
30-59 Days Delinquent      4209
60-89 Days Delinquent      2048
Name: count, dtype: int64

In [39]:
df['Liquidation/Prepayment Code'].value_counts()

Liquidation/Prepayment Code
Fully Paid, Prepaid       35849
Fully Paid, Matured        3529
Fully Paid, Refinanced      671
Foreclosure                 642
Third Party Sale             84
Other Liquidation            69
Dissolution                  41
Repurchase                   28
Deed-in-Lieu                 27
Discounted Payoff            12
Substitution                  1
Name: count, dtype: int64

In [23]:
df['Liquidation/Prepayment Code'].value_counts()

Liquidation/Prepayment Code
Fully Paid, Prepaid       35849
Fully Paid, Matured        3529
Fully Paid, Refinanced      671
Foreclosure                 642
Third Party Sale             84
Other Liquidation            69
Dissolution                  41
Repurchase                   28
Deed-in-Lieu                 27
Discounted Payoff            12
Substitution                  1
Name: count, dtype: int64

In [18]:
false_defaulters

[1673867585,
 1673887828,
 1673888928,
 1673908757,
 1673916783,
 1673946427,
 1673946429,
 1673946431,
 1673958629,
 1673966041,
 1674032877,
 1674032878,
 1674044964,
 1674047591,
 1674065825,
 1674086447,
 1674089571,
 1674107624,
 1674146231,
 1674156968,
 1674170534,
 1674199885,
 1674343233,
 1674352894,
 1674447501,
 1674451349,
 1674456062,
 1674470957,
 1674470958,
 1674478499,
 1674484870,
 1674494372,
 1674621761,
 1674653973,
 1674659743,
 1674663660,
 1674695227,
 1674715082,
 1674715083,
 1674752072,
 1674757754,
 1674768447,
 1674771953,
 1674777477,
 1674788628,
 1674835106,
 1674876515,
 1674880753,
 1674902069,
 1674989609,
 1675021925,
 1675028765,
 1675028766,
 1675051797,
 1675083135,
 1675120244,
 1675120248,
 1675129862,
 1675129863,
 1675129864,
 1675146259,
 1675161940,
 1675182391,
 1675199275,
 1675244426,
 1675259070,
 1675262794,
 1675279414,
 1675307697,
 1675378621,
 1675405789,
 1675435666,
 1675437817,
 1675441202,
 1675446847,
 1675475783,
 1675490353,

In [36]:
df['Loan Age'].value_counts(dropna=False)

Loan Age
 NaN      2205773
 120.0     285056
 117.0     153686
 118.0     125949
 114.0      99320
           ...   
 3.0           35
 2.0           17
-1.0            5
 0.0            4
 1.0            2
Name: count, Length: 249, dtype: int64

In [41]:
df[df['Loan Number'] == 1717462470]

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator
2009112,1717462470,2014-08-29,2014-07-24,2024-08-01,"$21,560,000.00",Interest Only/Amortizing/Balloon,Fixed,DUS,"$21,560,000.00",360.0,4.11,First,AM6435,2014-08-01,77.0,2.18,UW Actual DSCR,120,36.0,2017-08-01,Y,Standard DUS,NaN,1.0,172.0,Dedicated Student,1989,COLUMBIA,MO,65201,"COLUMBIA, MO METROPOLITAN STATISTICAL AREA",33.2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2015-02-01,1.0,4.11,2024-08-01,"$21,560,000.00",NaN,Current,N,NaN,N,NaN,"YM(114), See Issuance Documents(6)","YM(01/31/2024), See Issuance Documents(08/01/2...",NaN,NaN,NaN,Involuntary Prepayment,Standard,58.0,N,NaN
2009113,1717462470,2014-08-29,2014-07-24,2024-08-01,"$21,560,000.00",Interest Only/Amortizing/Balloon,Fixed,DUS,"$21,560,000.00",360.0,4.11,First,AM6435,2014-08-01,77.0,2.18,UW Actual DSCR,120,36.0,2017-08-01,Y,Standard DUS,NaN,1.0,172.0,Dedicated Student,1989,COLUMBIA,MO,65201,"COLUMBIA, MO METROPOLITAN STATISTICAL AREA",33.2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2016-05-01,1.0,4.11,2024-08-01,"$21,560,000.00",NaN,Current,N,NaN,N,NaN,"YM(114), See Issuance Documents(6)","YM(01/31/2024), See Issuance Documents(08/01/2...",NaN,NaN,NaN,Involuntary Prepayment,Standard,58.0,N,NaN
2009114,1717462470,2014-08-29,2014-07-24,2024-08-01,"$21,560,000.00",Interest Only/Amortizing/Balloon,Fixed,DUS,"$21,560,000.00",360.0,4.11,First,AM6435,2014-08-01,77.0,2.18,UW Actual DSCR,120,36.0,2017-08-01,Y,Standard DUS,NaN,1.0,172.0,Dedicated Student,1989,COLUMBIA,MO,65201,"COLUMBIA, MO METROPOLITAN STATISTICAL AREA",33.2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2018-12-01,1.0,4.11,2024-08-01,"$21,077,427.07",NaN,Current,N,NaN,N,NaN,"YM(114), See Issuance Documents(6)","YM(01/31/2024), See Issuance Documents(08/01/2...",NaN,NaN,NaN,Involuntary Prepayment,Standard,58.0,N,NaN
2009115,1717462470,2014-08-29,2014-07-24,2024-08-01,"$21,560,000.00",Interest Only/Amortizing/Balloon,Fixed,DUS,"$21,560,000.00",360.0,4.11,First,AM6435,2014-08-01,77.0,2.18,UW Actual DSCR,120,36.0,2017-08-01,Y,Standard DUS,NaN,1.0,172.0,Dedicated Student,1989,COLUMBIA,MO,65201,"COLUMBIA, MO METROPOLITAN STATISTICAL AREA",33.2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2015-01-01,1.0,4.11,2024-08-01,"$21,560,000.00",NaN,Current,N,NaN,N,NaN,"YM(114), See Issuance Documents(6)","YM(01/31/2024), See Issuance Documents(08/01/2...",NaN,NaN,NaN,Involuntary Prepayment,Standard,58.0,N,NaN
2009116,1717462470,2014-08-29,2014-07-24,2024-08-01,"$21,560,000.00",Interest Only/Amortizing/Balloon,Fixed,DUS,"$21,560,000.00",360.0,4.11,First,AM6435,2014-08-01,77.0,2.18,UW Actual DSCR,120,36.0,2017-08-01,Y,Standard DUS,NaN,1.0,172.0,Dedicated Student,1989,COLUMBIA,MO,65201,"COLUMBIA, MO METROPOLITAN STATISTICAL AREA",33.2,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,REO,2017-02-01,1.0,4.11,2024-08-01,"$21,560,000.00",NaN,Current,N,NaN,N,NaN,"YM(114)

In [8]:
df.head()

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator
8,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-03-01,1.0,7.649,2009-12-01,"$13,916,280.12",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN
9,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-04-01,1.0,7.650,2009-12-01,"$14,411,758.26",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN
10,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-12-01,1.0,7.649,2009-12-01,"$14,308,018.40",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN
11,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2002-07-01,1.0,7.649,2009-12-01,"$14,373,472.44",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN
12,1673867584,2000-01-01,1999-11-01,2009-12-01,"$14,725,385.83",Amortizing Balloon,Fixed,DUS,"$14,736,000.00",360.0,7.649,First,382113.0,2000-01-01,80.0,1.25,Lender UW DSCR,120,NaN,NaN,N,NaN,NaN,1.0,408.0,Multifamily,1985,ARLINGTON,TX,76017,"DALLAS-FORT WORTH-ARLINGTON, TX METROPOLITAN S...",92.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-02-01,1.0,7.650,2009-12-01,"$14,581,085.13",NaN,Current,N,NaN,N,NaN,"YM(114), O*(6)",NaN,NaN,NaN,NaN,Paid Prior to Yield Maintenance End Date,Standard,64.0,N,NaN


In [20]:
df['Final LP Date'] = pd.to_datetime(df['Final LP Date'])
df['Reporting Period Date'] = pd.to_datetime(df['Reporting Period Date'])

# Get the minimum Reporting Period Date per loan
min_reporting_date = df.groupby('Loan Number')['Reporting Period Date'].min()

# Get the unique Final LP Date per loan (non-null only)
final_lp_date = df.drop_duplicates('Loan Number')[['Loan Number', 'Final LP Date']].set_index('Loan Number')['Final LP Date']

# Filter for loans where Final LP Date < earliest Reporting Period Date
bad_loans = final_lp_date[final_lp_date < min_reporting_date]

# View the Loan Numbers
print(bad_loans.index.tolist())

[140296, 140297]


In [ ]:
### DROPPING 

#Drop the 2 loans where the final liquidation event date is before all the reporting dates 


Dropped this loan? False
✅ Dropped 2 loans where liquidation/prepayment occurred before any reporting period.


In [6]:
df

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator
0,140296,2000-10-31,1985-07-16,2001-08-10,"$82,501.71",NaN,ARM,DUS,"$82,501.71",NaN,5.196,First,NaN,NaN,50.0,NaN,NaN,360,NaN,NaN,Y,NaN,NaN,1.0,4.0,Multifamily,NaN,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-10-01,1.0,NaN,2001-08-10,"$82,501.71",NaN,Current,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,140296,2000-10-31,1985-07-16,2001-08-10,"$82,501.71",NaN,ARM,DUS,"$82,501.71",NaN,5.196,First,NaN,NaN,50.0,NaN,NaN,360,NaN,NaN,Y,NaN,NaN,1.0,4.0,Multifamily,NaN,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-11-01,1.0,NaN,2001-08-10,"$82,501.71",NaN,Current,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,140296,2000-10-31,1985-07-16,2001-08-10,"$82,501.71",NaN,ARM,DUS,"$82,501.71",NaN,5.196,First,NaN,NaN,50.0,NaN,NaN,360,NaN,NaN,Y,NaN,NaN,1.0,4.0,Multifamily,NaN,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,"Fully Paid, Prepaid",2000-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-01-01,1.0,5.61,2001-08-10,"$82,501.71",NaN,90+ Days Delinquent,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,140296,2000-10-31,1985-07-16,2001-08-10,"$82,501.71",NaN,ARM,DUS,"$82,501.71",NaN,5.196,First,NaN,NaN,50.0,NaN,NaN,360,NaN,NaN,Y,NaN,NaN,1.0,4.0,Multifamily,NaN,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000-12-01,1.0,NaN,2001-08-10,"$82,501.71",NaN,Current,N,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,140297,2000-10-31,1985-07-18,2001-08-10,"$548,872.98",NaN,ARM,DUS,"$548,872.98",NaN,5.196,First,NaN,NaN,50.0,NaN,NaN,360,NaN,NaN,Y,NaN,NaN,1.0,2.0,Multifamily,NaN,SAN DIEGO,CA,92116,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,"Fully Paid, Prepaid",2000-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-01-01,1.0,5.55,2000-08-10,"$205,587.07",NaN,90+ Days Delinquent,Y,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4960376,8300008176,2021-02-25,2021-02-25,2051-02-01,"$16,400,000.00",Interest Only/Amortizing/Balloon,Fixed,Non-DUS,"$16,400,000.00",480.0,4.130,First,NaN,NaN,62.4,1.18,UW DSCR NCF,360,24.0,2023-02-01,N,Pari Passu,NaN,1.0,102.0,Multifamily,1925,NEW YORK,NY,10027,"NEW YORK-NEWARK-JERSEY CITY, NY-NJ-PA METROPOL...",95.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-01,1.0,4.13,2051-02-01,"$16,221,752.36",NaN,Current,N,NaN,N,NaN,"5%(84), 4%(24), 3%(24), 2%(24), 1%(24), O*(180)",NaN,Project Based HAP/Sec 8,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [57]:
#Make a column with number of month through the loan info 

df['Acquisition Date'] = pd.to_datetime(df['Acquisition Date'])
df['Reporting Period Date'] = pd.to_datetime(df['Reporting Period Date'])

# Compute months since acquisition
df['Months Since Acquisition'] = (
    (df['Reporting Period Date'].dt.year - df['Acquisition Date'].dt.year) * 12 +
    (df['Reporting Period Date'].dt.month - df['Acquisition Date'].dt.month)
)

In [58]:
#Make binary for defaulting 
df['Default Flag'] = df['Loan Payment Status'].apply(
    lambda x: 0 if x == 'Current' else 1
)

In [59]:
df['Prepaid Flag'] = df['Liquidation/Prepayment Code'].apply(
    lambda x: 1 if x == 'Fully Paid, Prepaid' else 0
)

In [60]:
#Start of handling the fact that sometimes prepayments and stuff show up at the same time as default
#For now just 
df['Liquidation/Prepayment Date'] = pd.to_datetime(df['Liquidation/Prepayment Date'])
df = df.sort_values(by=['Loan Number', 'Liquidation/Prepayment Date'])
has_event = df[df['Liquidation/Prepayment Code'].notna()]
last_event_row = has_event.groupby('Loan Number').tail(1)
final_code_dict = last_event_row.set_index('Loan Number')['Liquidation/Prepayment Code'].to_dict()
final_date_dict = last_event_row.set_index('Loan Number')['Liquidation/Prepayment Date'].to_dict()
df['final_payment_event_code'] = df['Loan Number'].map(final_code_dict)
df['final_payment_event_date'] = df['Loan Number'].map(final_date_dict)

In [61]:
#Trying to fix the same thing still 
df['final_event_type'] = pd.NA
df['final_event_occurred'] = pd.NA
loans_to_drop = set()
loans_with_final = df[df['final_payment_event_date'].notna()]['Loan Number'].unique()
for loan in loans_with_final:
    sub = df[df['Loan Number'] == loan]
    event_date = sub['final_payment_event_date'].iloc[0]
    event_code = sub['final_payment_event_code'].iloc[0]

    # Find reporting range
    min_date = sub['Reporting Period Date'].min()
    max_date = sub['Reporting Period Date'].max()

    # If event outside reporting window → drop loan
    if event_date < min_date or event_date > max_date:
        loans_to_drop.add(loan)
        continue

    # Default all rows to 0 for this loan
    match = df['Loan Number'] == loan
    df.loc[match, 'final_event_occurred'] = 0

    # Find first row where Reporting Date ≥ event_date
    target_idx = sub[sub['Reporting Period Date'] >= event_date].sort_values('Reporting Period Date').index.min()

    if pd.notna(target_idx):
        df.loc[target_idx, 'final_event_type'] = event_code
        df.loc[target_idx, 'final_event_occurred'] = 1
    else:
        # shouldn't happen due to earlier check, but just in case
        loans_to_drop.add(loan)
df = df[~df['Loan Number'].isin(loans_to_drop)]
print(f"{len(loans_to_drop)} loans dropped due to final event date being outside of reporting window.")

40904 loans dropped due to final event date being outside of reporting window.


In [62]:
df.head(10)

,Loan Number,Acquisition Date,Note Date,Maturity Date at Acquisition,Loan Acquisition UPB,Amortization Type,Interest Type,Loan Product Type,Original UPB,Amortization Term,Original Interest Rate,Lien Position,Transaction ID,Issue Date,Loan Acquisition LTV,Underwritten DSCR,Underwritten DSCR Type,Original Term,Original I/O Term,I/O End Date,Loan Ever 60+ Days Delinquent,Loss Sharing Type,Modified Loss Sharing Percentage,Number of Properties at Acquisition,Property Acquisition Total Unit Count,Specific Property Type,Year Built,Property City,Property State,Property Zip Code,Metropolitan Statistical Area,Physical Occupancy %,Liquidation/Prepayment Code,Liquidation/Prepayment Date,Foreclosure Date,Credit Event Date,Foreclosure Value,Lifetime Net Credit Loss Amount,Sale Price,Default Amount,Credit Event Type,Reporting Period Date,Loan Active Property Count,Note Rate,Maturity Date - Current,UPB - Current,Delinquency UPB,Loan Payment Status,SDQ Indicator,Most Recent Modification Date,Modification Indicator,Defeasance Date,Prepayment Provision,Prepayment Provision End Date,Affordable Housing Type,MCIRT Deal ID,MCAS Deal ID,DUS Prepayment Outcomes,DUS Prepayment Segments,Loan Age,Green Bond Indicator,Social Bond Indicator,Months Since Acquisition,Default Flag,Prepaid Flag,final_payment_event_code,final_payment_event_date,final_event_type,final_event_occurred
11214,1674109600,2000-02-01,2000-02-14,2010-05-01,"$7,448,500.00",Amortizing Balloon,Fixed,DUS,"$7,448,500.00",360.0,8.28,First,382275.0,2000-02-01,72.0,1.25,Lender UW DSCR,122,NaN,NaN,N,Pari Passu,NaN,1.0,252.0,Multifamily,1974,DENVER,CO,80234,"DENVER-AURORA-LAKEWOOD, CO METROPOLITAN STATIS...",93.0,"Fully Paid, Matured",2010-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-05-01,1.0,8.28,2010-05-01,"$6,697,876.83",NaN,Current,N,NaN,N,NaN,"L(36), DEF(86)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,123,0,0,"Fully Paid, Matured",2010-04-30,"Fully Paid, Matured",1
11094,1674109600,2000-02-01,2000-02-14,2010-05-01,"$7,448,500.00",Amortizing Balloon,Fixed,DUS,"$7,448,500.00",360.0,8.28,First,382275.0,2000-02-01,72.0,1.25,Lender UW DSCR,122,NaN,NaN,N,Pari Passu,NaN,1.0,252.0,Multifamily,1974,DENVER,CO,80234,"DENVER-AURORA-LAKEWOOD, CO METROPOLITAN STATIS...",93.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-08-01,1.0,8.28,2010-05-01,"$7,378,184.84",NaN,Current,N,NaN,N,NaN,"L(36), DEF(86)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,18,0,0,"Fully Paid, Matured",2010-04-30,<NA>,0
11095,1674109600,2000-02-01,2000-02-14,2010-05-01,"$7,448,500.00",Amortizing Balloon,Fixed,DUS,"$7,448,500.00",360.0,8.28,First,382275.0,2000-02-01,72.0,1.25,Lender UW DSCR,122,NaN,NaN,N,Pari Passu,NaN,1.0,252.0,Multifamily,1974,DENVER,CO,80234,"DENVER-AURORA-LAKEWOOD, CO METROPOLITAN STATIS...",93.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-06-01,1.0,8.28,2010-05-01,"$7,386,815.04",NaN,Current,N,NaN,N,NaN,"L(36), DEF(86)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,16,0,0,"Fully Paid, Matured",2010-04-30,<NA>,0
11096,1674109600,2000-02-01,2000-02-14,2010-05-01,"$7,448,500.00",Amortizing Balloon,Fixed,DUS,"$7,448,500.00",360.0,8.28,First,382275.0,2000-02-01,72.0,1.25,Lender UW DSCR,122,NaN,NaN,N,Pari Passu,NaN,1.0,252.0,Multifamily,1974,DENVER,CO,80234,"DENVER-AURORA-LAKEWOOD, CO METROPOLITAN STATIS...",93.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-03-01,1.0,8.28,2010-05-01,"$6,706,177.05",NaN,Current,N,NaN,N,NaN,"L(36), DEF(86)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,121,0,0,"Fully Paid, Matured",2010-04-30,<NA>,0
11097,1674109600,2000-02-01,2000-02-14,2010-05-01,"$7,448,500.00",Amortizing Balloon,Fixed,DUS,"$7,448,500.00",360.0,8.28,First,382275.0,2000-02-01,72.0,1.25,Lender UW DSCR,122,NaN,NaN,N,Pari Passu,NaN,1.0,252.0,Multifamily,1974,DENVER,CO,80234,"DENVER-AURORA-LAKEWOOD, CO METROPOLITAN STATIS...",93.0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005-05-01,1.0,8.28,2010-05-01,"$7,141,925.73",NaN,Current,N,NaN,N,NaN,"L(36), DEF(86)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,63,0,0,"Fully Paid, Matured",2010-04-30,<NA>,0
11098,1674109600,2000-02-01,2000-02-14,2010-05-01,"

In [37]:
df['final_event_type'].value_counts()

final_event_type
Fully Paid, Prepaid    9
Other Liquidation      8
Third Party Sale       4
Foreclosure            3
Fully Paid, Matured    2
Discounted Payoff      1
Name: count, dtype: int64

In [14]:
count = df[(df['Any Default'] == 1) & (df['Prepaid Flag'] == 1)].shape[0]

In [15]:
count

238

In [16]:
count = df[(df['Any Default'] == 1) & (df['Prepaid Flag'] == 1)]['Loan Number'].nunique()
count

238

In [4]:
# Convert to datetime if not already
df['Liquidation/Prepayment Date'] = pd.to_datetime(df['Liquidation/Prepayment Date'], errors='coerce')
df['Reporting Period Date'] = pd.to_datetime(df['Reporting Period Date'], errors='coerce')

# Step 1: Get final liquidation/prepay date per loan
final_liq_date = df.sort_values('Reporting Period Date').dropna(subset=['Liquidation/Prepayment Date']) \
                   .groupby('Loan Number')['Liquidation/Prepayment Date'].last()

# Step 2: Get min and max reporting period date per loan
reporting_bounds = df.groupby('Loan Number')['Reporting Period Date'].agg(['min', 'max'])

# Step 3: Join both together
merged = reporting_bounds.join(final_liq_date)

# Step 4: Build conditions
before_min = merged['Liquidation/Prepayment Date'] < merged['min']
after_max = merged['Liquidation/Prepayment Date'] > merged['max']
outside_bounds = before_min | after_max

# Step 5: Count
count_total_outside = outside_bounds.sum()
count_before = before_min.sum()
count_after = after_max.sum()

print("Number of loans where final Liquidation/Prepayment Date falls outside reporting period range:", count_total_outside)
print("↳ Number where it's BEFORE the first reporting period:", count_before)
print("↳ Number where it's AFTER the last reporting period:", count_after)

Number of loans where final Liquidation/Prepayment Date falls outside reporting period range: 40904
↳ Number where it's BEFORE the first reporting period: 2
↳ Number where it's AFTER the last reporting period: 40902


In [5]:
after_max = merged['Liquidation/Prepayment Date'] > merged['max']
after_dates = merged[after_max].copy()

# Step 2: Calculate how far after (in days)
after_dates['Days After Max'] = (after_dates['Liquidation/Prepayment Date'] - after_dates['max']).dt.days

# Step 3: Summary statistics
print("Summary of how far after the reporting period the liquidation date is:")
print(after_dates['Days After Max'].describe())

# Optional: see how many are just a few days late
bins = [0, 1, 7, 14, 30, 90, 365, float('inf')]
labels = ['1 day', '1 week', '2 weeks', '1 month', '3 months', '1 year', 'over 1 year']
after_dates['Bucket'] = pd.cut(after_dates['Days After Max'], bins=bins, labels=labels, right=True)

print("\nBreakdown by how far after:")
print(after_dates['Bucket'].value_counts().sort_index())

Summary of how far after the reporting period the liquidation date is:
count    40902.000000
mean        50.802235
std         11.486515
min          1.000000
25%         44.000000
50%         57.000000
75%         60.000000
max         88.000000
Name: Days After Max, dtype: float64

Breakdown by how far after:
Bucket
1 day             17
1 week            87
2 weeks          107
1 month         3968
3 months       36723
1 year             0
over 1 year        0
Name: count, dtype: int64
